<a href="https://colab.research.google.com/github/fgsantosti/ProgramacaoDispositivosMoveisFlutter/blob/main/App_Fluuter_14_Clean_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto To-Do List em Flutter com Clean Architecture

## Descrição do Projeto

Este projeto é uma aplicação de lista de tarefas (To-Do List) desenvolvida utilizando Flutter. A aplicação segue os princípios da Clean Architecture, garantindo uma separação clara de responsabilidades e uma estrutura modular. O projeto está dividido em três camadas principais: **Data**, **Domain** e **Presentation**, cada uma com suas próprias responsabilidades.

## Estrutura do Projeto

```
lib/
├── core/
│   ├── error/
│   └── usecases/
├── features/
│   └── task/
│       ├── data/
│       │   ├── datasources/
│       │   │   └── task_local_data_source.dart
│       │   ├── models/
│       │   │   └── task_model.dart
│       │   └── repositories/
│       │       └── task_repository_impl.dart
│       ├── domain/
│       │   ├── entities/
│       │   │   └── task.dart
│       │   ├── repositories/
│       │   │   └── task_repository.dart
│       │   └── usecases/
│       │       ├── add_task.dart
│       │       ├── get_tasks.dart
│       │       ├── update_task.dart
│       │       └── delete_task.dart
│       └── presentation/
│           ├── bloc/
│           │   ├── task_bloc.dart
│           │   ├── task_event.dart
│           │   └── task_state.dart
│           ├── pages/
│           │   └── task_page.dart
│           └── widgets/
│               └── task_list_widget.dart
└── main.dart
```

## Explicação dos Arquivos

### `core/`

#### `error/`

Este diretório é reservado para gerenciar classes de erro e exceções que podem ser compartilhadas entre diferentes recursos do projeto.

#### `usecases/`

Neste diretório, mantemos as definições genéricas de casos de uso que podem ser reutilizados.

### `features/task/`

#### `data/`

##### `datasources/task_local_data_source.dart`

Este arquivo define a interface `TaskLocalDataSource` e sua implementação `TaskLocalDataSourceImpl`. É responsável por simular a persistência de dados localmente em uma lista de tarefas.

```dart
abstract class TaskLocalDataSource {
  Future<List<TaskModel>> getTasks();
  Future<void> addTask(TaskModel task);
  Future<void> updateTask(TaskModel task);
  Future<void> deleteTask(String id);
}

class TaskLocalDataSourceImpl implements TaskLocalDataSource {
  List<TaskModel> tasks = [];

  @override
  Future<List<TaskModel>> getTasks() async {
    return tasks;
  }

  @override
  Future<void> addTask(TaskModel task) async {
    tasks.add(task);
  }

  @override
  Future<void> updateTask(TaskModel task) async {
    final index = tasks.indexWhere((t) => t.id == task.id);
    if (index != -1) {
      tasks[index] = task;
    }
  }

  @override
  Future<void> deleteTask(String id) async {
    tasks.removeWhere((task) => task.id == id);
  }
}
```

##### `models/task_model.dart`

Este arquivo define o modelo de dados `TaskModel` que extende a entidade `Task`. Ele é responsável pela conversão de dados entre o formato usado na camada de dados e o formato usado na camada de domínio.

```dart
class TaskModel extends Task {
  TaskModel({
    required String id,
    required String title,
    bool isCompleted = false,
  }) : super(id: id, title: title, isCompleted: isCompleted);

  factory TaskModel.fromEntity(Task task) {
    return TaskModel(
      id: task.id,
      title: task.title,
      isCompleted: task.isCompleted,
    );
  }

  Task toEntity() {
    return Task(
      id: id,
      title: title,
      isCompleted: isCompleted,
    );
  }
}
```

##### `repositories/task_repository_impl.dart`

Este arquivo implementa o repositório `TaskRepository`. Ele atua como um intermediário entre a camada de dados e a camada de domínio.

```dart
class TaskRepositoryImpl implements TaskRepository {
  final TaskLocalDataSource localDataSource;

  TaskRepositoryImpl({required this.localDataSource});

  @override
  Future<List<Task>> getTasks() async {
    final taskModels = await localDataSource.getTasks();
    return taskModels.map((taskModel) => taskModel.toEntity()).toList();
  }

  @override
  Future<void> addTask(Task task) async {
    final taskModel = TaskModel.fromEntity(task);
    return await localDataSource.addTask(taskModel);
  }

  @override
  Future<void> updateTask(Task task) async {
    final taskModel = TaskModel.fromEntity(task);
    return await localDataSource.updateTask(taskModel);
  }

  @override
  Future<void> deleteTask(String id) async {
    return await localDataSource.deleteTask(id);
  }
}
```

#### `domain/`

##### `entities/task.dart`

Define a entidade `Task` que representa o modelo de dados principal na camada de domínio.

```dart
class Task extends Equatable {
  final String id;
  final String title;
  final bool isCompleted;

  Task({
    required this.id,
    required this.title,
    this.isCompleted = false,
  });

  @override
  List<Object?> get props => [id, title, isCompleted];
}
```

##### `repositories/task_repository.dart`

Define a interface `TaskRepository` que declara as operações que podem ser realizadas em uma tarefa.

```dart
abstract class TaskRepository {
  Future<List<Task>> getTasks();
  Future<void> addTask(Task task);
  Future<void> updateTask(Task task);
  Future<void> deleteTask(String id);
}
```

##### `usecases/add_task.dart`

Define o caso de uso para adicionar uma nova tarefa.

```dart
class AddTask {
  final TaskRepository repository;

  AddTask(this.repository);

  Future<void> call(Task task) async {
    await repository.addTask(task);
  }
}
```

##### `usecases/get_tasks.dart`

Define o caso de uso para obter a lista de tarefas.

```dart
class GetTasks {
  final TaskRepository repository;

  GetTasks(this.repository);

  Future<List<Task>> call() async {
    return await repository.getTasks();
  }
}
```

##### `usecases/update_task.dart`

Define o caso de uso para atualizar uma tarefa existente.

```dart
class UpdateTask {
  final TaskRepository repository;

  UpdateTask(this.repository);

  Future<void> call(Task task) async {
    await repository.updateTask(task);
  }
}
```

##### `usecases/delete_task.dart`

Define o caso de uso para deletar uma tarefa.

```dart
class DeleteTask {
  final TaskRepository repository;

  DeleteTask(this.repository);

  Future<void> call(String id) async {
    await repository.deleteTask(id);
  }
}
```

#### `presentation/`

##### `bloc/task_bloc.dart`

Define a lógica do BLoC (Business Logic Component) para gerenciar o estado da aplicação.

```dart
class TaskBloc extends Bloc<TaskEvent, TaskState> {
  final GetTasks getTasks;
  final AddTask addTask;
  final UpdateTask updateTask;
  final DeleteTask deleteTask;

  TaskBloc({
    required this.getTasks,
    required this.addTask,
    required this.updateTask,
    required this.deleteTask,
  }) : super(TaskInitial()) {
    on<LoadTasks>((event, emit) async {
      emit(TaskLoading());
      try {
        final tasks = await getTasks();
        emit(TaskLoaded(tasks));
      } catch (e) {
        emit(TaskError(e.toString()));
      }
    });

    on<AddNewTask>((event, emit) async {
      await addTask(event.task);
      add(LoadTasks());
    });

    on<UpdateExistingTask>((event, emit) async {
      await updateTask(event.task);
      add(LoadTasks());
    });

    on<DeleteExistingTask>((event, emit) async {
      await deleteTask(event.id);
      add(LoadTasks());
    });
  }
}
```

##### `bloc/task_event.dart`

Define os eventos que podem ocorrer na aplicação relacionados às tarefas.

```dart
abstract class TaskEvent extends Equatable {
  @override
  List<Object?> get props => [];
}

class LoadTasks extends TaskEvent {}

class AddNewTask extends TaskEvent {
  final Task task;

  AddNewTask(this.task);

  @override
  List<Object?> get props => [task];
}

class UpdateExistingTask extends TaskEvent {
  final Task task;

  UpdateExistingTask(this.task);

  @override
  List<Object?> get props => [task];
}

class DeleteExistingTask extends TaskEvent {
  final String id;

  DeleteExistingTask(this.id);

  @override
  List<Object?> get props => [id];
}
```

##### `bloc/task_state.dart`

Define os possíveis estados da aplicação relacionados às tarefas.

```dart
abstract class TaskState extends Equatable {
  @override
  List<Object?> get props => [];
}

class TaskInitial extends TaskState {}

class TaskLoading extends TaskState {}

class TaskLoaded extends TaskState {
  final List<Task> tasks;

  TaskLoaded(this.tasks);

  @override
  List<Object?> get props => [tasks];
}

class TaskError extends TaskState {
  final String message;

  TaskError(this.message);



  @override
  List<Object?> get props => [message];
}
```

##### `pages/task_page.dart`

Define a página principal da aplicação onde as tarefas são exibidas e gerenciadas.

```dart
class TaskPage extends StatefulWidget {
  @override
  _TaskPageState createState() => _TaskPageState();
}

class _TaskPageState extends State<TaskPage> {
  final TextEditingController _taskController = TextEditingController();

  void _showAddTaskDialog(BuildContext context) {
    showDialog(
      context: context,
      builder: (BuildContext context) {
        return AlertDialog(
          title: Text('Add Task'),
          content: TextField(
            controller: _taskController,
            decoration: InputDecoration(hintText: 'Enter task title'),
          ),
          actions: <Widget>[
            TextButton(
              child: Text('Cancel'),
              onPressed: () {
                _taskController.clear();
                Navigator.of(context).pop();
              },
            ),
            TextButton(
              child: Text('Add'),
              onPressed: () {
                final task = Task(id: DateTime.now().toString(), title: _taskController.text);
                context.read<TaskBloc>().add(AddNewTask(task));
                _taskController.clear();
                Navigator.of(context).pop();
              },
            ),
          ],
        );
      },
    );
  }

  void _showUpdateTaskDialog(BuildContext context, Task task) {
    _taskController.text = task.title;
    showDialog(
      context: context,
      builder: (BuildContext context) {
        return AlertDialog(
          title: Text('Update Task'),
          content: TextField(
            controller: _taskController,
            decoration: InputDecoration(hintText: 'Enter new task title'),
          ),
          actions: <Widget>[
            TextButton(
              child: Text('Cancel'),
              onPressed: () {
                _taskController.clear();
                Navigator.of(context).pop();
              },
            ),
            TextButton(
              child: Text('Update'),
              onPressed: () {
                final updatedTask = Task(id: task.id, title: _taskController.text, isCompleted: task.isCompleted);
                context.read<TaskBloc>().add(UpdateExistingTask(updatedTask));
                _taskController.clear();
                Navigator.of(context).pop();
              },
            ),
          ],
        );
      },
    );
  }

  void _confirmDeleteTask(BuildContext context, String taskId) {
    showDialog(
      context: context,
      builder: (BuildContext context) {
        return AlertDialog(
          title: Text('Delete Task'),
          content: Text('Are you sure you want to delete this task?'),
          actions: <Widget>[
            TextButton(
              child: Text('Cancel'),
              onPressed: () {
                Navigator.of(context).pop();
              },
            ),
            TextButton(
              child: Text('Delete'),
              onPressed: () {
                context.read<TaskBloc>().add(DeleteExistingTask(taskId));
                Navigator.of(context).pop();
              },
            ),
          ],
        );
      },
    );
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('To-Do App'),
      ),
      body: BlocBuilder<TaskBloc, TaskState>(
        builder: (context, state) {
          if (state is TaskLoading) {
            return Center(child: CircularProgressIndicator());
          } else if (state is TaskLoaded) {
            return TaskListWidget(
              tasks: state.tasks,
              onUpdateTask: (task) => _showUpdateTaskDialog(context, task),
              onDeleteTask: (taskId) => _confirmDeleteTask(context, taskId),
            );
          } else if (state is TaskError) {
            return Center(child: Text(state.message));
          }
          return Container();
        },
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () => _showAddTaskDialog(context),
        child: Icon(Icons.add),
      ),
    );
  }
}
```

##### `widgets/task_list_widget.dart`

Define um widget que exibe a lista de tarefas com opções para atualizar e deletar.

```dart
class TaskListWidget extends StatelessWidget {
  final List<Task> tasks;
  final Function(Task) onUpdateTask;
  final Function(String) onDeleteTask;

  TaskListWidget({
    required this.tasks,
    required this.onUpdateTask,
    required this.onDeleteTask,
  });

  @override
  Widget build(BuildContext context) {
    return ListView.builder(
      itemCount: tasks.length,
      itemBuilder: (context, index) {
        final task = tasks[index];
        return ListTile(
          title: Text(task.title),
          trailing: Row(
            mainAxisSize: MainAxisSize.min,
            children: [
              IconButton(
                icon: Icon(Icons.edit),
                onPressed: () => onUpdateTask(task),
              ),
              IconButton(
                icon: Icon(Icons.delete),
                onPressed: () => onDeleteTask(task.id),
              ),
            ],
          ),
        );
      },
    );
  }
}
```

### `main.dart`

Define o ponto de entrada da aplicação e configura as dependências necessárias.

```dart
void main() {
  final TaskLocalDataSource localDataSource = TaskLocalDataSourceImpl();
  final TaskRepository repository = TaskRepositoryImpl(localDataSource: localDataSource);
  final GetTasks getTasks = GetTasks(repository);
  final AddTask addTask = AddTask(repository);
  final UpdateTask updateTask = UpdateTask(repository);
  final DeleteTask deleteTask = DeleteTask(repository);

  runApp(MyApp(
    taskBloc: TaskBloc(
      getTasks: getTasks,
      addTask: addTask,
      updateTask: updateTask,
      deleteTask: deleteTask,
    ),
  ));
}

class MyApp extends StatelessWidget {
  final TaskBloc taskBloc;

  MyApp({required this.taskBloc});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Flutter To-Do List',
      home: BlocProvider(
        create: (_) => taskBloc..add(LoadTasks()),
        child: TaskPage(),
      ),
    );
  }
}
```

## Conclusão

Este projeto demonstra como aplicar os princípios da Clean Architecture em uma aplicação Flutter. Cada camada da arquitetura é separada, facilitando a manutenção, testes e escalabilidade do código. Utilizamos BLoC para gerenciar o estado da aplicação, garantindo que a lógica de negócio esteja isolada da interface do usuário. Este exemplo é ideal para ser utilizado para ilustrar boas práticas de desenvolvimento de software.

## Estrutura do Projeto

```
lib/
├── core/
│   ├── error/
│   └── usecases/
├── features/
│   └── task/
│       ├── data/
│       │   ├── datasources/
│       │   │   └── task_local_data_source.dart
│       │   ├── models/
│       │   │   └── task_model.dart
│       │   └── repositories/
│       │       └── task_repository_impl.dart
│       ├── domain/
│       │   ├── entities/
│       │   │   └── task.dart
│       │   ├── repositories/
│       │   │   └── task_repository.dart
│       │   └── usecases/
│       │       ├── add_task.dart
│       │       ├── get_tasks.dart
│       │       ├── update_task.dart
│       │       └── delete_task.dart
│       └── presentation/
│           ├── bloc/
│           │   ├── task_bloc.dart
│           │   ├── task_event.dart
│           │   └── task_state.dart
│           ├── pages/
│           │   └── task_page.dart
│           └── widgets/
│               └── task_list_widget.dart
└── main.dart
```
